<a href="https://colab.research.google.com/github/PacktPublishing/Hands-On-Computer-Vision-with-PyTorch/blob/master/Chapter02/Specifying_batch_size_while_training_a_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 2.3.1 数据集、数据加载器和批大小

In [1]:
# 1. 导入用于加载数据和处理数据集的方法:
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn

In [2]:
# 2. 导入数据, 将数据转换为浮点数, 并注册到设备

# 提供需要处理的数据点:
x = [[1,2],[3,4],[5,6],[7,8]]
y = [[3],[7],[11],[15]]

In [3]:
# 将数据转换成浮点数
X = torch.tensor(x).float()
Y = torch.tensor(y).float()

In [4]:
# 将数据注册到设备上
device = 'cuda' if torch.cuda.is_available() else 'cpu'
X = X.to(device)
Y = Y.to(device)

In [5]:
# 3. 实例化一个数据集类 MyDataset:

# 在MyDataset类中, 存储信息每次获取一个数据点, 以便可以将一批数据点捆绑在一起(使用DataLoader),
# 并通过一个前向和一个反向传播发送, 以更新权重
class MyDataset(Dataset):
    # 定义一个__init__方法, 用于接收输入和输出对, 并将它们转换为Torch浮点对象:
    def __init__(self, x, y):
        self.x = torch.tensor(x).float()
        self.y = torch.tensor(y).float()
    # 指定输入数据集的长度(__len__):
    def __len__(self):
        return len(self.x)
    # 最后, 用__getitem__方法获取特定的行
    def __getitem__(self, ix):
        # ix指的是需要从数据集中获取的行的索引
        return self.x[ix], self.y[ix]

# 4. 创建已定义类的实例:
ds = MyDataset(X, Y)

C:\Users\86183\AppData\Local\Temp\ipykernel_27508\4126783974.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.x = torch.tensor(x).float()
C:\Users\86183\AppData\Local\Temp\ipykernel_27508\4126783974.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.y = torch.tensor(y).float()


In [6]:
# 5. 通过DataLoader传递之前定义的数据集实例, 获取原始输入和输出张量对象
dl = DataLoader(ds, batch_size=2, shuffle=True)

# 在上述代码中, 还指定从原始数据集(ds)中获取两个数据点(通过batch_size=2)的一个随机样本(通过shuffle=True)
# 为了从dl中获取批数据, 需要进行如下循环
for x, y in dl:
    print(x, y)

tensor([[3., 4.],
        [7., 8.]], device='cuda:0') tensor([[ 7.],
        [15.]], device='cuda:0')
tensor([[5., 6.],
        [1., 2.]], device='cuda:0') tensor([[11.],
        [ 3.]], device='cuda:0')


In [7]:
# 6. 定义一个神经网络类
class MyNeuralNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.input_to_hidden_layer = nn.Linear(2,8)
        self.hidden_layer_activation = nn.ReLU()
        self.hidden_to_output_layer = nn.Linear(8,1)
    def forward(self, x):
        x = self.input_to_hidden_layer(x)
        x = self.hidden_layer_activation(x)
        x = self.hidden_to_output_layer(x)
        return x

In [8]:
# 7. 定义模型对象、损失函数和优化器
mynet = MyNeuralNet().to(device)
loss_func = nn.MSELoss()
from torch.optim import SGD
opt = SGD(mynet.parameters(), lr = 0.001)

In [9]:
import time

# 8. 循环遍历各批数据点以最小化损失值, 正如前一节第6步所做的一样
loss_history = []
start = time.time()
for _ in range(50):
    for data in dl:
        x, y = data
        opt.zero_grad()
        loss_value = loss_func(mynet(x),y)
        loss_value.backward()
        opt.step()
        loss_history.append(loss_value)
end = time.time()
print(end - start)

1.5108284950256348


### 2.3.2 预测新的数据点

In [10]:
# 1. 创建用于测试模型的数据点:
val_x = [[10,11]]

In [11]:
# 2. 将新数据点转换为一个张量浮点对象并注册到设备:
val_x = torch.tensor(val_x).float().to(device)

In [12]:
# 3. 前向传播
mynet(val_x)


tensor([[20.4343]], device='cuda:0', grad_fn=<AddmmBackward0>)